In [73]:
#dw13 หน้าแรก
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = 'https://www.thaiwarrant.com/dw/search'
head = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'

print('0')

html = requests.get(url, verify=False, headers={'User-Agent': head}).content
df_list = pd.read_html(html)
df_dw13 = df_list[2]
df_dw13.columns.values[0] = 'Unname'
df_dw13.columns.values[1] = 'DW'
df_dw13.columns.values[2] = 'Type'
df_dw13.columns.values[9] = 'TTM(days)'
df_dw13 = df_dw13.drop(df_dw13.index[-1])

0


c:\Users\Inv_edg\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaiwarrant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [74]:
#dw13 หน้า 2
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
# Set up Chrome WebDriver (make sure you have downloaded and placed chromedriver in your PATH)
driver = webdriver.Chrome()

# Open the webpage
driver.get("https://www.thaiwarrant.com/dw/search")

try:
    # Find the link element
    link = driver.find_element(By.XPATH, "//a[contains(@href, '__doPostBack') and text()='2']")

    # Click the link using JavaScript
    driver.execute_script("arguments[0].click();", link)
    time.sleep(10) #ถ้ามันขึ้นdattaframeเหมือนอันแรกต้อง time.sleep นานขึ้น
    html_after_click = driver.page_source
    df_list = pd.read_html(html_after_click)
    df_dw132 = df_list[2]
except Exception as e:
    print("Error:", e)

# Optionally, you can do further actions after clicking the link

# Close the browser
#driver.quit()
df_dw132.columns.values[0] = 'Unname'
df_dw132.columns.values[1] = 'DW'
df_dw132.columns.values[2] = 'Type'
df_dw132.columns.values[9] = 'TTM(days)'

C:\Users\Inv_edg\AppData\Local\Temp\ipykernel_16568\2539153144.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(html_after_click)


In [75]:
result_df = pd.concat([df_dw13, df_dw132], ignore_index=True)

In [76]:
imp_vol = []
for i in range(len(result_df)):
    volatility = 0
    dw = result_df['DW'][i]
    url = "https://www.thaiwarrant.com/dw/" + dw
    response = requests.get(url, verify=False, headers={'User-Agent': head}).content

    soup = BeautifulSoup(response, 'html.parser')
    inner_text = soup.get_text()
    match = re.search(r"Volatility \(%\) :\s*([\d.]+)", inner_text)
    if match:
        volatility = match.group(1)
    else:
        volatility = 'NaN'
    imp_vol.append(volatility)
result_df['Imp Vol.'] = np.array(imp_vol)
result_df = result_df.drop(['Unname'],axis=1)

c:\Users\Inv_edg\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaiwarrant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Inv_edg\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaiwarrant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Inv_edg\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaiwarrant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/

In [77]:
from datetime import datetime
today_date = datetime.today().strftime('%Y-%m-%d')

file_name = f'DW13_{today_date}.xlsx'
result_df.to_excel(file_name, index=False)